## RAG Architecture

A brief introduction and demo

## LangChain
LangChain is a framework for developing applications powered by language models. 

It's very useful to build RAG pipelines

In [2]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/unstructured/pdfs/Fly devs.pdf")
data = loader.load()
print(f"This document has {len(data)} pages")

This document has 2 pages


### Text splitters

We need to break Documents into smaller chunks because is useful both, for indexing data and passing it to a model, large chunks are harder to search over and won't fit in model's context window.

It's recommended chunks between 500 and 1000 characters.

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    length_function=len,
    chunk_overlap=50
)

documents = text_splitter.split_documents(data)
print(f"Now we have {len(documents)} documents!")

Now we have 5 documents!


Generate Embeddings

In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Instantiate OpenAI Embeddings
embedding_openai = OpenAIEmbeddings(model = "text-embedding-ada-002")

# Create Vector Store with Embeddings
vector_store = Chroma.from_documents(
    documents = documents,
    embedding = embedding_openai
)

Let's test it with a query

In [4]:
vector_store = Chroma(
    persist_directory = "../data/data_embeddings/attention-embeddings",
    embedding_function = embedding_openai
)

In [6]:
query = "What are transformers?"

docs = vector_store.similarity_search_with_score(query, k=5)
docs

[(Document(page_content='Minimum Viable Product (MVPs) to enable faster time to market, attract early adopters, \nand achieve product -market fit from early on. The most trustworthy companies and \npowerful start -ups had partnered with us to get real value, run operations efficiently, and \ngain flexibility. Our goal is to enhance the growth of your business by establishing a \nsuccessful partnership . \n \nWhat we do  at fly devs  \nInnovative and unique software solutions, custom -tailored to the needs of your business.', metadata={'page': 0, 'source': '../data/unstructured/pdfs/Fly devs.pdf'}),
  0.5469162464141846),
 (Document(page_content='Minimum Viable Product (MVPs) to enable faster time to market, attract early adopters, \nand achieve product -market fit from early on. The most trustworthy companies and \npowerful start -ups had partnered with us to get real value, run operations efficiently, and \ngain flexibility. Our goal is to enhance the growth of your business by establ

In [8]:
query = "What is attention?"

docs = vector_store.similarity_search_with_score(query, k=5)
docs

[(Document(page_content='reduced to a constant number of operations, albeit at the cost of reduced effective resolution due\nto averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as\ndescribed in section 3.2.\nSelf-attention, sometimes called intra-attention is an attention mechanism relating different positions\nof a single sequence in order to compute a representation of the sequence. Self-attention has been', metadata={'page': 1, 'source': '../data/unstructured/pdfs/atenttion_is_all_you_need.pdf'}),
  0.39198678731918335),
 (Document(page_content='reduced to a constant number of operations, albeit at the cost of reduced effective resolution due\nto averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as\ndescribed in section 3.2.\nSelf-attention, sometimes called intra-attention is an attention mechanism relating different positions\nof a single sequence in order to compute a representation of the sequence. 

Let's create a retriever

In [5]:
retriever = vector_store.as_retriever(
    search_kwargs = {"k":4}
)
query = "What are transformers?"
retriever.get_relevant_documents(query)

/home/josealcocer27/.cache/pypoetry/virtualenvs/pycon-colombia-2024-rDJfgf35-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='Minimum Viable Product (MVPs) to enable faster time to market, attract early adopters, \nand achieve product -market fit from early on. The most trustworthy companies and \npowerful start -ups had partnered with us to get real value, run operations efficiently, and \ngain flexibility. Our goal is to enhance the growth of your business by establishing a \nsuccessful partnership . \n \nWhat we do  at fly devs  \nInnovative and unique software solutions, custom -tailored to the needs of your business.', metadata={'page': 0, 'source': '../data/unstructured/pdfs/Fly devs.pdf'}),
 Document(page_content='Product development : Connect with Flydevs for the opportunity to receive a customized \nsoftware product that addresses your specific business needs.  \nIT Staff Outsourcing : Build a global technology partnership with the help of our project \noutsourcing service to allocate resources cost -effectively and focus on complete \ntransparency.  \nFlydevs has top -notch I

In [6]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain

In [7]:
llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0.0,
    streaming=True
)

qa_chain_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents=True

)

Finally we can create a Chain to ask question based on our base knowledge (Vector Store)

In [8]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0.0
)

qa_chain_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents=True,
    verbose = True
)

/home/josealcocer27/.cache/pypoetry/virtualenvs/pycon-colombia-2024-rDJfgf35-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Let's use our RAG pipeline 😎

In [14]:
qa_chain_with_sources.invoke({"question":"What are transformers?"})



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.


{'question': 'What are transformers?',
 'answer': 'Transformers are models that rely entirely on self-attention to compute representations of input and output without using sequence-aligned RNNs or convolution.\n',
 'sources': '../data/unstructured/pdfs/atenttion_is_all_you_need.pdf',
 'source_documents': [Document(page_content='The Transformer allows for significantly more parallelization and can reach a new state of the art in\ntranslation quality after being trained for as little as twelve hours on eight P100 GPUs.\n2 Background\nThe goal of reducing sequential computation also forms the foundation of the Extended Neural GPU\n[16], ByteNet [ 18] and ConvS2S [ 9], all of which use convolutional neural networks as basic building', metadata={'page': 1, 'source': '../data/unstructured/pdfs/atenttion_is_all_you_need.pdf'}),
  Document(page_content='The Transformer allows for significantly more parallelization and can reach a new state of the art in\ntranslation quality after being traine

In [9]:
import os
import pandas as pd
import glob
from typing import Dict, Any
import streamlit as st
from chromadb.api.models.Collection import Collection
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.vectorstores.chroma import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.base import BaseConversationalRetrievalChain
from langchain.prompts import SystemMessagePromptTemplate
from langchain.memory import ConversationBufferMemory

In [10]:
combine_docs_template_customer_service = """You are a Customer Service Agent from the {company} company and solve questions about the context provided. You can answer common questions
as how are you and things that people use in normal conversations, but when you receive questions about products, services or something else you have 
to answer only based in the context provided, so you can introduce your self as Customer Service Agent from the {company} company but only when someone asks who are you.
Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------

{context}

Question: {question}
Helpful Answer:"
"""

template_customer_service = '''
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know.
Don't try to make up an answer. The context could be in any language, but you will always answer in the language of the user's question.
You have to act as Customer Service Agent from the HugggingFace company and solve questions about the context provided. You can answer common questions
as how are you and thins that people use in normal conversations, but when you receive questions about products, services or something else you have 
to answer only based in the context provided.

Context: {context}

Question: {question}
Answer:
'''


template_customer_service2 = '''
You have to act as Customer Service Agent from the {company} company and solve questions about the context provided. You can answer common questions
as how are you and thins that people use in normal conversations, but when you receive questions about products, services or something else you have 
to answer only based in the context provided.

'''

In [11]:
from langchain.callbacks.streaming_aiter import AsyncIteratorCallbackHandler
from langchain.callbacks.streaming_stdout_final_only import FinalStreamingStdOutCallbackHandler

In [12]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0.0,
    streaming=True
)
condense_question_llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0.0
)
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    verbose=True,
    # memory=memory,
    return_generated_question=False,
    retriever=retriever,
    condense_question_llm = condense_question_llm,
    #condense_question_prompt=PromptTemplate.from_template(custom_template),
)
qa.combine_docs_chain.llm_chain.prompt.messages[0] = SystemMessagePromptTemplate.from_template(combine_docs_template_customer_service.replace("{company}", "Fly devs"))

In [8]:
qa.run("Introduce yourself!")

/home/josealcocer27/.cache/pypoetry/virtualenvs/pycon-colombia-2024-rDJfgf35-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Customer Service Agent from the Fly devs company and solve questions about the context provided. You can answer common questions
as how are you and things that people use in normal conversations, but when you receive questions about products, services or something else you have 
to answer only based in the context provided, so you can introduce your self as Customer Service Agent from the Fly devs company but only when someone asks who are you.
Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------

Product development : Connect with Flydevs for the opportunity to receive a customized 
software product that addresses your specific business needs.  
IT Staff Outsourcing : Build a global technology partnership with the help of our project 
outsour

'Hello! I am a Customer Service Agent from the Fly devs company. How can I assist you today?'

In [13]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# chat completion llm
llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

In [14]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.invoke,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    )
]

In [15]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory = memory,
    get_chat_history=lambda h : h
)

/home/josealcocer27/.cache/pypoetry/virtualenvs/pycon-colombia-2024-rDJfgf35-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [16]:
template_customer_service2 = """
Assistant is a large language model trained by {company}.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Assistant have to act as Customer Service Agent from the {company} company and solve questions about the context provided. He cans answer common questions
as how are you and things that people use in normal conversations.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""

In [17]:
agent.agent.llm_chain.prompt.messages[0].prompt.template = template_customer_service2.replace("{company}", "Fly Devs")

In [18]:
from langchain.callbacks.streaming_aiter import AsyncIteratorCallbackHandler
from langchain.callbacks.streaming_stdout_final_only import FinalStreamingStdOutCallbackHandler
from langchain.schema import LLMResult

class AsyncCallbackHandler(AsyncIteratorCallbackHandler):
    content: str = ""
    final_answer: bool = False
    
    def __init__(self) -> None:
        super().__init__()

    async def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
        self.content += token
        # if we passed the final answer, we put tokens in queue
        if self.final_answer:
            if '"action_input": "' in self.content:
                if token not in ['"', "}"]:
                    self.queue.put_nowait(token)
        elif "Final Answer" in self.content:
            self.final_answer = True
            self.content = ""
    
    async def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        if self.final_answer:
            self.content = ""
            self.final_answer = False
            self.done.set()
        else:
            self.content = ""

In [19]:
import asyncio

async def run_call(query: str, stream_it: AsyncCallbackHandler):
    # assign callback handler
    agent.agent.llm_chain.llm.callbacks = [stream_it]
    # now query
    await agent.acall(inputs={"input": query})

async def create_gen(query: str, stream_it: AsyncCallbackHandler):
    task = asyncio.create_task(run_call(query, stream_it))
    async for token in stream_it.aiter():
        yield token
    await task


stream_it = AsyncCallbackHandler()
gen = create_gen("Who are you", stream_it)

In [20]:
chunks = []
for chunk in gen:
    chunks.append(chunk)
    print("------")
    print(chunk)

TypeError: 'async_generator' object is not iterable

In [36]:
agent.astream({"input": "Where is Fly Devs located?"})

<async_generator object AgentExecutor.astream at 0x7f371709c280>

In [17]:
agent.invoke({"input": "Where is Fly Devs located?"})



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "I am Assistant, a large language model trained by Fly Devs to assist with a wide range of tasks and provide information on various topics."
}
```

> Finished chain.


{'input': 'who are you?',
 'chat_history': [HumanMessage(content='Wher is Fly Devs locatead?'),
  AIMessage(content='Fly Devs has locations in Argentina, Spain, the United States, and Australia.'),
  HumanMessage(content='who are you?'),
  AIMessage(content='I am Assistant, a large language model trained by Fly Devs to assist with a wide range of tasks and provide information on various topics.')],
 'output': 'I am Assistant, a large language model trained by Fly Devs to assist with a wide range of tasks and provide information on various topics.'}

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("../data/unstructured/pdfs/atenttion_is_all_you_need.pdf")
data = loader.load()
print(f"This document has {len(data)} pages")


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    length_function=len,
    chunk_overlap=50
)

documents = text_splitter.split_documents(data)
print(f"Now we have {len(documents)} documents!")

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Instantiate OpenAI Embeddings
embedding_openai = OpenAIEmbeddings(model = "text-embedding-ada-002")

# Create Vector Store with Embeddings
vector_store = Chroma.from_documents(
    documents = documents,
    embedding = embedding_openai
)
retriever = vector_store.as_retriever()

This document has 15 pages
Now we have 93 documents!


In [5]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"documents_search",
                      "Search for information about attention is all you need!")

retriever_tool.name

tools=[retriever_tool]

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [8]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [14]:
prompt.messages[0].

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant'))

In [9]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [10]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [11]:

agent_executor.invoke({"input":"What is attention is all you need?"})



> Entering new AgentExecutor chain...

Invoking: `documents_search` with `{'query': 'attention is all you need'}`


but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>Figure 4: Two attention heads, also in layer 5 of 6, apparently involved in anaphora resolution. Top:
Full attentions for head 5. Bottom: Isolated attentions from just the word ‘its’ for attention heads 5
and 6. Note that the attentions are very sharp for this word.
14

but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>Figure 4: Two attention heads, also in layer 5 of 6, apparently involved in anaphora resolution. Top:
Full attentions for head 5. Bottom: Isolated attentions from just the word ‘its’ for attention heads 5
and 6. Note that the attentions are very sharp for this word.
14

but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>Figure 5: Many of the attention heads exhibit behaviour th

{'input': 'What is attention is all you need?',
 'output': '"Attention is all you need" seems to be related to a concept in a document discussing attention heads and their application in anaphora resolution. The document mentions that the application of attention should be just what is missing. It also refers to attention heads exhibiting behavior related to the structure of sentences. If you need more detailed information or clarification, please let me know.'}